In [ ]:
import numpy as np
import neal
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

#------------------------------------------Solve 200 Q matrices using Neal-------------------------------------------
random = np.random
random.seed(42)
#Define the size of the Q matrix (n x n)
num_instances = 200 
num_variables = 20   
# Create a simulated annealer
sampler = neal.SimulatedAnnealingSampler()
instance_results = []
for instance_id in range(num_instances):
    Q_matrix = np.random.randint(-10, 10, size=(num_variables, num_variables))   
    Q_matrix = (Q_matrix + Q_matrix.T) // 2                                      # Generate a symmetric QUBO matrix
    qubo_dict = {(i, j): Q_matrix[i, j] for i in range(num_variables) for j in range(i, num_variables)}
    # Solve the QUBO problem using Simulated Annealing
    response = sampler.sample_qubo(qubo_dict, num_reads=500)
    # Extract the optimal solution for this instance
    optimal_sample = min(response.data(['sample', 'energy']), key=lambda x: x.energy)
    # Store results (instance_id, optimal energy, optimal solution, Q_matrix)
    instance_results.append((instance_id, optimal_sample.energy, optimal_sample.sample, Q_matrix))    
for instance_id, optimal_energy, optimal_solution,_ in instance_results:
    print(f"\n Instance {instance_id + 1}:")
    print(f"   Optimal Energy: {optimal_energy}")
    print("   Optimal Solution:")
    for i, value in optimal_solution.items():
        print(f"   x[{i}] = {value}")
print("\n--- Q Matrix for Instance 1 ---")
Q_matrix_instance_1 = instance_results[0][3]  
print(Q_matrix_instance_1)

#-----------------------------------------Label edges as 1/-1---------------------------------------------------------
data = []
for instance_id, optimal_energy, optimal_solution,_ in instance_results:
    for i in sorted(optimal_solution.keys()):
        label = 1 if optimal_solution[i] == 1 else -1  # Label 1 for optimal solutions, -1 otherwise
        data.append((instance_id + 1, i, optimal_solution[i], optimal_energy, label))
df_qubo_label = pd.DataFrame(data, columns=["Instance", "Variable", "Value", "OptimalEnergy", "Label"])
#-----------------------------------Compute f1, f2, f3 scores for all variables----------------------------------------------
def compute_qubo_features(instance_num, Q_matrix):
    n = len(Q_matrix)  
    f1_values, f2_values, f3_values = {}, {}, {}  
    Q_diag = np.diag(Q_matrix)                                    # Extract min/max diagonal values for normalization
    q_min, q_max = np.min(Q_diag), np.max(Q_diag)    
    total_weight = np.sum(np.abs(Q_matrix))                       # Compute total interaction strength
    non_zero_elements = np.count_nonzero(Q_matrix)                # Compute total non-zero elements
    for i in range(n):
        key = (instance_num, i)  
        f1_values[key] = np.round(Q_matrix[i, i] / np.sum(np.abs(Q_diag)),3)
        f2_values[key] = np.round((Q_matrix[i, i] - q_min) / (q_max - q_min),3) if q_max != q_min else 0
        f3_values[key] = np.round((np.sum(np.abs(Q_matrix[i, :])) / total_weight),3)
    return f1_values, f2_values, f3_values
f1_scores, f2_scores, f3_scores = {}, {}, {}
for instance_num, _, _, Q_matrix in instance_results:              # Compute features for all instances
    f1, f2, f3= compute_qubo_features(instance_num, Q_matrix)
    f1_scores.update(f1)
    f2_scores.update(f2)
    f3_scores.update(f3)
f1_df = pd.DataFrame(f1_scores.items(), columns=["Instance_Variable", "f1"])
f2_df = pd.DataFrame(f2_scores.items(), columns=["Instance_Variable", "f2"])
f3_df = pd.DataFrame(f3_scores.items(), columns=["Instance_Variable", "f3"])
for df in [f1_df, f2_df, f3_df]:
    df[['Instance', 'Variable']] = pd.DataFrame(df['Instance_Variable'].tolist(), index=df.index)
    df.drop(columns=["Instance_Variable"], inplace=True)
final_features_df = f1_df.merge(f2_df, on=['Instance', 'Variable'])
final_features_df = final_features_df.merge(f3_df, on=['Instance', 'Variable'])
combined_QUBO_df = final_features_df.merge(df_qubo_label, on=['Instance', 'Variable'], how='inner')
#------------------------------------------Train the SVM model---------------------------------------------------
# Extract features and labels 
X = combined_QUBO_df[['f1', 'f2', 'f3']]        # Feature columns
y = combined_QUBO_df['Label']                   # Target labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train an SVM model with a linear kernel
svm_model_QUBO = SVC(kernel='linear', probability=True, random_state=42)
svm_model_QUBO.fit(X_train, y_train)
y_pred_QUBO = svm_model_QUBO.predict(X_test)
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_QUBO)
classification_rep = classification_report(y_test, y_pred_QUBO)
print(f"Model Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_rep)
#----------------------------------------------penalize misclassifying----------------------------------------
svm_model_QUBO2 = SVC(kernel='linear', probability=True, class_weight='balanced', random_state=42)
svm_model_QUBO2.fit(X_train, y_train)
y_pred_QUBO2 = svm_model_QUBO2.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_QUBO2)
classification_rep = classification_report(y_test, y_pred_QUBO2)
print(f"Model Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_rep)
#------------------------------------- Extract weight vector (w) and bias (b)-------------------------------------------
w_new= svm_model_QUBO2.coef_[0]   
b_new = svm_model_QUBO2.intercept_[0]  
print("Extracted Model Parameters:")
print(f"Weight Vector (w): {w_new}")
print(f"Bias (b): {b_new}")
#------------------------------------- Apply a new unseen Q matrix for test instance------------------------------------------------------
Q = pd.read_csv('Desktop/Q_matrix.csv', header=None).values

num_instances = 1  
Q_matrix_test_instances = []
for instance_id in range(num_instances):  
    Q_matrix_test_instances.append((instance_id + 1, Q)) 
#-----------------------------------Compute f1, f2, f3 scores for all variables in new Q matrix----------------------------------------------
f1_scores, f2_scores, f3_scores = {}, {}, {}
for instance_num, Q in Q_matrix_test_instances:
    f1, f2, f3= compute_qubo_features(instance_num, Q)
    f1_scores.update(f1)
    f2_scores.update(f2)
    f3_scores.update(f3)
f1_df = pd.DataFrame(f1_scores.items(), columns=["Instance_Variable", "f1"])
f2_df = pd.DataFrame(f2_scores.items(), columns=["Instance_Variable", "f2"])
f3_df = pd.DataFrame(f3_scores.items(), columns=["Instance_Variable", "f3"])
for df in [f1_df, f2_df, f3_df]:
    df[['Instance', 'Variable']] = pd.DataFrame(df['Instance_Variable'].tolist(), index=df.index)
    df.drop(columns=["Instance_Variable"], inplace=True)
final_features_df = f1_df.merge(f2_df, on=['Instance', 'Variable'])
final_features_test_df = final_features_df.merge(f3_df, on=['Instance', 'Variable'])
f_test_QUBO= final_features_test_df[['f1', 'f2', 'f3']].values 
w_new_column = w_new.reshape(-1, 1)  

#-------------------------------------Compute Z scalars------------------------------------------------------
z_values = np.dot(f_test_QUBO,w_new_column) + b_new                           # (z = f_test * w^T + b)
#------------------------------------- Compute predicted probability values for all variales in test instance------------------------------------
p_values = 1 / (1 + np.exp(-z_values))                                        # sigmoid function (p = 1 / (1 + e^(-z)))
z_values = z_values.flatten()
p_values = p_values.flatten()
p_values_df = pd.DataFrame({
    'Instance': final_features_test_df['Instance'].values,  
    'Variable': final_features_test_df['Variable'].values,  
    'p': p_values  
})
z_p_df = pd.DataFrame({
    'Instance': final_features_test_df['Instance'],
    'Variable': final_features_test_df['Variable'],
    'z': z_values,
    'p': p_values
})
final_features_test_df = final_features_test_df.drop(columns=['z', 'p'], errors='ignore')

final_combined_QUBO_df = final_features_test_df[['Instance', 'Variable', 'f1', 'f2', 'f3']].merge(
    z_p_df, on=['Instance', 'Variable'], how='left'
)

#----------------------------------------- Classical ACO algorithm for QUBO-------------------------------------------------
# This Classical ACO algorithm for QUBOis provided by Pramuditha
np.random.seed(42)
# Parameters
M = 100                             # Total number of artificial ants
MaxT =1000                          # Maximum number of iterations
alpha = 75                          # Pheromone exponential weight
beta = 75                           # Pheromone heuristic weight
rho = 0.5                           # Pheromone evaporation rate
q = 1                               # Pheromone deposit
Tau_0 =1                            # Initial pheromone level

b=Q
N=Q.shape[0]
Q = (b + b.T) / 2
P = np.full((N, 2), Tau_0, dtype=int)
# calculate M2
m2=0
for i in range(N):
    n_sum=0
    for j in range(i):
        if Q[i,j]<0:
            n_sum+=2*Q[i,j]
    if Q[i,i]<0:
        n_sum+=Q[i,i]
    m2+=n_sum
M2=1-m2
print(M2)

# Functions
def calculate_probabilities(current_index, solution, pheromones, Q_matrix, alpha, beta,M2):
    probabilities = []
    for j in [0, 1]:
        pheromone = pheromones[current_index, j] ** alpha
        visibility = (
            1 / (M2 + Q_matrix[current_index, current_index]*j +
                 2 * sum([solution[k] * Q_matrix[current_index, k]*j for k in range(len(solution))])))** beta
        probabilities.append(pheromone * visibility)
    probabilities_sum = sum(probabilities)
    return [prob / probabilities_sum for prob in probabilities]
def construct_solution(pheromones, Q_matrix, alpha, beta, N,M2):
    solution = []
    current_index = 0
    for _ in range(N):
        probabilities = calculate_probabilities(current_index, solution, pheromones, Q_matrix, alpha, beta,M2)
        next_choice = np.random.choice([0, 1], p=probabilities)
        solution.append(next_choice)
        current_index += 1  # Move to the next index
    return solution
def calculate_cost(solution, Q_matrix):
    solution = np.array(solution).reshape(-1, 1)
    c =np.dot(Q_matrix,solution)
    cost=np.dot(solution.T,c)
    return cost
def update_pheromones(pheromones, solutions, costs, evaporation_rate, pheromone_deposit,M2):
    pheromones = (1 - evaporation_rate)* pheromones  # Evaporate pheromones
    for solution, cost in zip(solutions, costs):
        for i in range(len(solution)):
            if solution[i] == 1:
                pheromones[i, 1] =pheromones[i, 1].item()+ pheromone_deposit /(M2+ cost)
            elif solution[i] == 0:
                pheromones[i, 0] =pheromones[i, 0].item()+ pheromone_deposit /(M2+ cost)
# Main loop
best_solution = None                  # Best solution found
best_cost = float('inf')              # Best cost found
Best_Solution =[]
ant_path=[[]*i for i in range(M)]
for t in range(MaxT):
    solutions = []                    # List to store solutions
    costs = []                        # List to store corresponding costs
    for a in range(M):                # a: number of ants
        solution = construct_solution(P, Q, alpha, beta, N,M2)
        cost = calculate_cost(solution, Q)
        solutions.append(solution)
        costs.append(cost)
        ant_path[a].append(cost)
        if cost < best_cost:          # Update the best solution
            best_solution = solution
            best_cost = cost
    update_pheromones(P, solutions, costs, rho, q,M2)
    print(f"Iteration {t + 1}, Best Cost: {best_cost}")
    Best_Solution.append(best_cost)

print("Best Solution:", best_solution)
print("Best Cost:", best_cost)
print(calculate_cost(best_solution,Q))
i=0
for j in  best_solution:
    print(i,j)
    i+=1

    print()
iteration=[t for t in range(MaxT)]

#-------------------------------------------- Ml-ACO algorithm for QUBO-------------------------------------------
np.random.seed(42)
# Parameters
M = 100                                   # Total number of artificial ants
MaxT =1000                                # Maximum number of iterations
alpha = 75                                # Pheromone exponential weight
beta = 75                                 # Pheromone heuristic weight
rho = 0.5                                 # Pheromone evaporation rate
q = 1                                     # Pheromone deposit
Tau_0 =1                                  # Initial pheromone level

# Problem definition

b=Q
N=Q.shape[0]
Q = (b + b.T) / 2
P = np.full((N, 2), Tau_0, dtype=int)
# calculate M2
m2=0
for i in range(N):
    n_sum=0
    for j in range(i):
        if Q[i,j]<0:
            n_sum+=2*Q[i,j]
    if Q[i,i]<0:
        n_sum+=Q[i,i]
    m2+=n_sum
M2=1-m2
print(M2)
p_values_dict = final_combined_QUBO_df.set_index('Variable')['p'].to_dict()
# Functions
def calculate_probabilities(current_index, solution, pheromones, Q_matrix, alpha, beta,M2,p_values_dict):
    probabilities = []
    for j in [0, 1]:
        pheromone = pheromones[current_index, j] ** alpha
        visibility = (
            1 / (M2 + Q_matrix[current_index, current_index]*j +
                 2 * sum([solution[k] * Q_matrix[current_index, k]*j for k in range(len(solution))])))** beta
        
        p_factor = p_values_dict[i] if j == 1 else (1 - p_values_dict[i])      
        visibility *= p_factor 
        probabilities.append(pheromone * visibility)
    probabilities_sum = sum(probabilities)
    return [prob / probabilities_sum for prob in probabilities]

def construct_solution(pheromones, Q_matrix, alpha, beta, N,M2,p_values_dict):
    solution = []
    current_index = 0
    for _ in range(N):
        probabilities = calculate_probabilities(current_index, solution, pheromones, Q_matrix, alpha, beta,M2,p_values_dict)
        next_choice = np.random.choice([0, 1], p=probabilities)
        solution.append(next_choice)
        current_index += 1  # Move to the next index
    return solution

def calculate_cost(solution, Q_matrix):
    solution = np.array(solution).reshape(-1, 1)
    c =np.dot(Q_matrix,solution)
    cost=np.dot(solution.T,c)
    return cost

def update_pheromones(pheromones, solutions, costs, evaporation_rate, pheromone_deposit,M2):
    pheromones = (1 - evaporation_rate)* pheromones  # Evaporate pheromones
    for solution, cost in zip(solutions, costs):
        for i in range(len(solution)):
            if solution[i] == 1:
                pheromones[i, 1] =pheromones[i, 1].item()+ pheromone_deposit /(M2+ cost)
            elif solution[i] == 0:
                pheromones[i, 0] =pheromones[i, 0].item()+ pheromone_deposit /(M2+ cost)


# Main loop
best_solution = None                  # Best solution found
best_cost = float('inf')              # Best cost found
Best_Solution =[]
ant_path=[[]*i for i in range(M)]
for t in range(MaxT):
    solutions = []                    # List to store solutions
    costs = []                        # List to store corresponding costs
    for a in range(M):                # a: number of ants
        solution = construct_solution(P, Q, alpha, beta, N,M2,p_values_dict)
        cost = calculate_cost(solution, Q)
        solutions.append(solution)
        costs.append(cost)
        ant_path[a].append(cost)
        if cost < best_cost:          # Update the best solution
            best_solution = solution
            best_cost = cost
    update_pheromones(P, solutions, costs, rho, q,M2)
    print(f"Iteration {t + 1}, Best Cost: {best_cost}")
    Best_Solution.append(best_cost)

print("Best Solution:", best_solution)
print("Best Cost:", best_cost)
print(calculate_cost(best_solution,Q))
i=0
for j in  best_solution:
    print(i,j)
    i+=1

    print()
iteration=[t for t in range(MaxT)]